In [65]:
import pandas as pd
import numpy as np

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.subplots import make_subplots

In [66]:
url1 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,health,count(health)' +\
        '&$group=boroname,health').replace(' ', '%20')

url2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=steward,health,count(health)' +\
        '&$group=steward,health').replace(' ', '%20')

borough_health = pd.read_json(url1)

df = borough_health.pivot(index='boroname',columns='health',values='count_health')
df

health,NaN,Fair,Good,Poor
boroname,,,,
Bronx,0,10887,66603,3095
Brooklyn,0,25073,138212,6459
Manhattan,0,11460,47358,3609
Queens,0,34549,194008,9417
Staten Island,0,14535,82669,4238


In [67]:
#df.iloc[0].sum()
#df.iloc[0,1]=df.iloc[0,1]/df.iloc[0].sum()
df = pd.DataFrame(df)

for i in range(0,5):
    tot = df.iloc[i].sum()
    for j in range(1,4):
        df = df.replace(df.iloc[i,j], round(df.iloc[i,j]/tot,2))
        
df = df.sort_values(by=['Poor', 'Fair'])
df

health,NaN,Fair,Good,Poor
boroname,,,,
Bronx,0,0.14,0.83,0.04
Staten Island,0,0.14,0.81,0.04
Brooklyn,0,0.15,0.81,0.04
Queens,0,0.15,0.82,0.04
Manhattan,0,0.18,0.76,0.06


In [68]:
steward_health = pd.read_json(url2)
steward_health

,steward,health,count_health
0,1or2,Fair,21902
1,1or2,Good,115372
2,1or2,Poor,6283
3,3or4,Fair,2830
4,3or4,Good,15606
5,3or4,Poor,747
6,4orMore,Fair,200
7,4orMore,Good,1361
8,4orMore,Poor,49
9,None,Fair,71572


In [69]:
df2 = steward_health.pivot(index='steward',columns='health',values='count_health')
df2

health,NaN,Fair,Good,Poor
steward,,,,
NaN,0.0,NaN,NaN,NaN
1or2,NaN,21902.0,115372.0,6283.0
3or4,NaN,2830.0,15606.0,747.0
4orMore,NaN,200.0,1361.0,49.0
None,0.0,71572.0,396511.0,19739.0


In [70]:
df2 = df2[['Poor','Fair','Good']]
df2 = df2.loc[['None','1or2','3or4','4orMore']]
df2

health,Poor,Fair,Good
steward,,,
None,19739.0,71572.0,396511.0
1or2,6283.0,21902.0,115372.0
3or4,747.0,2830.0,15606.0
4orMore,49.0,200.0,1361.0


In [71]:
for i in range(0,4):
    tot = df2.iloc[i].sum()
    for j in range(0,3):
        df2 = df2.replace(df2.iloc[i,j], round(df2.iloc[i,j]/tot,4))

df2 = df2.sort_index()
df2.drop(df2.tail(1).index,inplace=True)

In [72]:
from IPython import display
import os
def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = None):
    in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ if in_binder is None else in_binder
    if in_binder:
        base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
        url = 'https://hub.mybinder.org{}'.format(base_prefix)
        app.config.requests_pathname_prefix = base_prefix
    else:
        url = 'http://localhost:%d' % port
        
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)

In [80]:
app = dash.Dash()

app.layout = html.Div(children=[
    html.H1(children='NYC Tree Health'),

    html.Div(children='''
        Dash: Data608 Module 3 - Examining NYC Tree Health Using Dash App.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': df.index, 'y': df.Good, 'type': 'bar', 'name': 'Good'},
                {'x': df.index, 'y': df.Fair, 'type': 'bar', 'name': 'Fair'},
                {'x': df.index, 'y': df.Poor, 'type': 'bar', 'name': 'Poor'}
            ],
            'layout': {
                'title': 'NYC Tree Condition by Borough'
            }
        }
    ),
    dcc.Graph(
        id='example-graph-2',
        figure={
            'data': [
                {'x': df2.index, 'y': df2.Good, 'type': 'scatter', 'name': 'Good'}
            ],
            'layout': {
                'title': 'Steward Health Versus Good Tree Health'
            }
        }
    ),
    dcc.Graph(
        id='example-graph-3',
        figure={
            'data': [
                {'x': df2.index, 'y': df2.Fair, 'type': 'scatter', 'name': 'Fair'}
            ],
            'layout': {
                'title': 'Steward Health Versus Fair Tree Health'
            }
        }
    ),
    dcc.Graph(
        id='example-graph-4',
        figure={
            'data': [
                {'x': df2.index, 'y': df2.Poor, 'type': 'scatter', 'name': 'Poor'}
            ],
            'layout': {
                'title': 'Steward Impact Versus Poor Tree Health'
            }
        }
    )
])

In [ ]:
show_app(app)

Open in new window

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2020 15:48:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 15:48:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 15:48:06] "GET /_dash-dependencies HTTP/1.1" 200 -
